# 🧠 SynthMed Development Notebook
This Jupyter Notebook helps you develop, test, and package a **Watsonx Orchestrate Action** using the **Action Development Kit (ADK)**.

Follow the steps below to set up and test your custom action locally.

## 1. Environment Setup
Install dependencies and verify your environment.

In [1]:
!pip install ibm-watsonx-orchestrate pyyaml

## 2. Import Required Libraries
Load the necessary Python packages for ADK and YAML handling.

In [5]:
import requests
import subprocess
import yaml
from google.colab import userdata

## 3. Create and Activate SynthMed Environment
Set up the Orchestrate environment using user-defined API_URL and API_KEY.

In [6]:
def run_orchestrate_command(command_args):
  """
  Executes an 'orchestrate' CLI command and returns the output.

  Args:
      command_args (list): A list of strings representing the 'orchestrate'
                           command and its arguments. For example:
                           ['tools', 'import', '-k', 'python', '-f', 'my_tool.py']

  Returns:
      tuple: A tuple containing (stdout, stderr, return_code).
  """

  try:
    # Prepend 'orchestrate' to the command arguments
    full_command = ['orchestrate'] + command_args

    # Execute the command
    process = subprocess.run(
        full_command,
        capture_output=True,
        text=True,  # Decode stdout and stderr as text
        check=True   # Raise CalledProcessError for non-zero exit codes
    )
    return process.stdout, process.stderr, process.returncode
  except subprocess.CalledProcessError as e:
    print(f"Error executing command: {e}")
    print(f"Stdout: {e.stdout}")
    print(f"Stderr: {e.stderr}")
    return e.stdout, e.stderr, e.returncode
  except FileNotFoundError:
    print("Error: 'orchestrate' command not found. Ensure the watsonx Orchestrate ADK is installed and in your PATH.")
    return "", "Error: 'orchestrate' command not found.", 1

stdout, stderr, return_code = run_orchestrate_command(['env', 'remove', '-n', 'synthmed'])
if return_code == 0:
  print("Environment synthmed removed successfully.")
else:
  print("Environment sysnthmed did not exist.")

apiurl = userdata.get("API_URL")
apikey = userdata.get("API_KEY")

stdout, stderr, return_code = run_orchestrate_command(
  ['env', 'add', '-n', 'synthmed', '-u', apiurl, '--type', 'ibm_iam'])

if return_code == 0:
  print("Environment synthmed added successfully.")
else:
  print("Environment synthmed not added successfully.")

stdout, stderr, return_code = run_orchestrate_command(['env', 'activate', '-a', apikey, 'synthmed'])
if return_code == 0:
  print("Environment synthmed activated successfully.")
else:
  print("Environment synthmed not activated successfully")

Environment synthmed removed successfully.
Environment synthmed added successfully.
Environment synthmed activated successfully.


## 4. Test Your Action Logic
Import and test the Python function that defines your action.

In [ ]:
from my_action.main import my_function  # 👈 replace with your actual function

test_input = {
    'param1': 'value1',
    'param2': 'value2'
}

result = my_function(test_input)
print('Action result:', result)

## 5. Simulate Orchestrate Execution
Use the ADK ActionRunner to run your action as Orchestrate would.

In [ ]:
runner = ActionRunner(action_path=action_path)
runner.run()

## 6. Package the Action for Upload
Once validated, package your action into a `.zip` for uploading to Watsonx Orchestrate.

In [ ]:
!adk package $action_path

✅ **Next Steps:**
- Log in to the [Watsonx Orchestrate Developer Portal](https://watsonx.ibm.com/orchestrate)
- Upload the generated `.zip` package
- Test it in the sandbox or production workspace